In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!unrar x "/content/drive/MyDrive/dataset_v3_yolo.rar" "/content/dataset/"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/dataset_v3_yolo_pf.rar

Creating    /content/dataset                                          OK
Extracting  /content/dataset/cpd.yaml                                      0%  OK 
Creating    /content/dataset/test                                     OK
Extracting  /content/dataset/test/104.jpg                                  0%  OK 
Extracting  /content/dataset/test/104.txt                                  0%  OK 
Extracting  /content/dataset/test/109.jpg                                  0%  OK 
Extracting  /content/dataset/test/109.txt                                  0%  OK 
Extracting  /content/dataset/test/117.jpg                                  0%  OK 
Extracting  /content/dataset/test/117.txt                                  0%  OK 
Extracting  /content/dataset/test/156.jpg                              

In [ ]:
!ls dataset

cpd.yaml  test	train  validation


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.5 MB/s eta 0:00:00


In [ ]:
!rm -r /content/runs/train

rm: cannot remove '/content/runs/train': No such file or directory


In [ ]:
!yolo detect train data=/content/dataset/cpd.yaml model=yolo26n.pt epochs=300 imgsz=640 patience=35 batch=64 save_period=30

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/cpd.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=35, perspective=0.0

In [ ]:
!rm -r /content/runs/detect/train2

## Test

In [ ]:
!yolo detect val data=/content/dataset/cpd.yaml model=/content/runs/detect/train3/weights/best.pt split=test

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
YOLO26n summary (fused): 122 layers, 2,375,421 parameters, 0 gradients, 5.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 104.8±156.5 MB/s, size: 81.2 KB)
val: Scanning /content/dataset/test... 170 images, 18 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 170/170 706.2it/s 0.2s
val: New cache created: /content/dataset/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 1.8it/s 6.2s
                   all        170        269      0.942      0.862      0.926      0.658
                 adult         68         84      0.946      0.829      0.953      0.691
                 child         59        109      0.923      0.877      0.924      0.624
                   pet         70         76      0.957      0.878      0.901      0.658
Speed: 5.6ms preprocess, 8.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to

## Download

In [ ]:
import shutil
from google.colab import files

# 1. Zip the folder (replace 'folder_name' with your folder's name)
# This creates 'results.zip' from the folder '/content/runs/detect/train5'
shutil.make_archive('yolo_v26n640pf', 'zip', '/content/runs/detect')

'/content/yolo_v26n640pf.zip'

In [ ]:
files.download('yolo_v26n640pf.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from ultralytics import YOLO
model = YOLO('/content/runs/detect/train3/weights/best.pt')
model.export(format='tflite',
    int8=True,
    imgsz=640)

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO26n summary (fused): 122 layers, 2,375,421 parameters, 0 gradients, 5.2 GFLOPs

PyTorch: starting from '/content/runs/detect/train3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 6) (5.1 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx>=1.12.0,<2.0.0', 'onnx2tf>=1.26.3,<1.29.0', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 7.60s
Prepared 9 packages in 1.54s
Installed 9 packages in 377ms
 + ai-edge-litert==2.1.2
 + backports-strenum==1.3.1
 + colorama==0.4.6


/usr/local/lib/python3.12/dist-packages/torch/onnx/_internal/torchscript_exporter/utils.py:1447: OnnxExporterWarning: Exporting to ONNX opset version 22 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 20. To use a newer opset version, consider 'torch.onnx.export(..., dynamo=True)'. 
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/onnx/_internal/torchscript_exporter/symbolic_opset9.py:5353: UserWarning: Exporting aten::index operator of advanced indexing in opset 22 is achieved by combination of multiple ONNX operators, including Reshape, Transpose, Concat, and Gather. If indices include negative values, the exported graph will produce incorrect results.
  warnings.warn(


ONNX: slimming with onnxslim 0.1.85...
ONNX: export success ✅ 1.9s, saved as '/content/runs/detect/train3/weights/best.onnx' (9.4 MB)
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100% ━━━━━━━━━━━━ 1/1 51.6files/s 0.0s
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.8...
Saved artifact at '/content/runs/detect/train3/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 640, 640, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 300, 6), dtype=tf.float32, name=None)
Captures:
  134709406614736: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  134709406613776: TensorSpec(shape=(3, 3, 3, 16), dtype=tf.float32, name=None)
  134709406614160: TensorSpec(shape=(16,), dtype=tf.float32, name=None)
  134709406618192: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)

'/content/runs/detect/train3/weights/best_saved_model/best_int8.tflite'

In [ ]:
!rm -r /content/runs/detect/val2

## best full integer quantization test

In [ ]:
!yolo detect val data=/content/dataset/cpd.yaml model=/content/runs/detect/train/weights/best_saved_model/best_full_integer_quant.tflite split=test imgsz=480

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
E0000 00:00:1770931418.972437   71031 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770931418.979545   71031 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770931418.997799   71031 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770931418.997826   71031 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770931418.997830   71031 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770931418.997833   71031

## Float 32 quantization test

In [ ]:
!yolo detect val data=/content/dataset/cpd.yaml model=/content/runs/detect/train3/weights/best_saved_model/best_float32.tflite split=test imgsz=640

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
E0000 00:00:1771009051.909998   28742 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771009051.917248   28742 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771009051.935452   28742 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009051.935479   28742 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009051.935484   28742 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009051.935487   28742

## Float 16

In [ ]:
!yolo detect val data=/content/dataset/cpd.yaml model=/content/runs/detect/train3/weights/best_saved_model/best_float16.tflite split=test imgsz=640

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
E0000 00:00:1771009106.516144   28997 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771009106.523179   28997 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771009106.540918   28997 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009106.540947   28997 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009106.540951   28997 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009106.540956   28997

## int_8

In [ ]:
!yolo detect val data=/content/dataset/cpd.yaml model=/content/runs/detect/train3/weights/best_saved_model/best_int8.tflite split=test imgsz=640

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
E0000 00:00:1771009216.787600   29517 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771009216.794423   29517 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771009216.811435   29517 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009216.811466   29517 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009216.811470   29517 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009216.811475   29517

## best_int_quant

In [ ]:
!yolo detect val data=/content/dataset/cpd.yaml model=/content/runs/detect/train3/weights/best_saved_model/best_integer_quant.tflite split=test imgsz=640

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
E0000 00:00:1771009174.143085   29314 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771009174.149804   29314 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771009174.167081   29314 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009174.167109   29314 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009174.167113   29314 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771009174.167118   29314